In [5]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path
from datetime import datetime, timezone, timedelta

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import librosa

from sklearn.metrics import roc_auc_score, average_precision_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import json
import timm

from importlib import reload

logging.basicConfig(level=logging.ERROR)

from module import preprocess_lib, datasets_lib, utils_lib, models_lib, learning_lib, config_lib
reload(config_lib)

<module 'module.config_lib' from '/root/program/birdclef-2025/scripts/module/config_lib.py'>

In [6]:


class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = '/kaggle/input/birdclef-2025-0330'
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.RAW_DIR = '../data/raw/'
            self.PROCESSED_DIR = '../data/processed/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.models_dir = "../models/" # 全modelの保存先
            self.model_path = self.models_dir # 各モデルの保存先．学習時に動的に変更．
            
            self.spectrogram_npy = '../data/processed/baseline/birdclef2025_melspec_5sec_256_256.npy'
            
            self.pseudo_label_csv = "../data/result/pseudo_labels_baseline_7sec.csv"
            self.pseudo_melspec_npy = "../data/processed/train_soundscapes_0407/train_soundscapes_melspecs.npy"

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5.0 # 推論時のウィンドウサイズ
        self.TARGET_DURATION = 5.0 # データセット作成時のウィンドウサイズ
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 128
        self.FMIN = 50
        self.FMAX = 14000        

        # ===== Training Mode =====
        if mode == "train":
            self.seed = 42
            self.apex = False
            self.print_freq = 100
            self.num_workers = 2

            self.LOAD_DATA = True
            self.epochs = 10
            self.batch_size = 32
            self.criterion = 'BCEWithLogitsLoss'

            self.n_fold = 5
            self.selected_folds = [0, 1, 2, 3, 4]

            self.optimizer = 'AdamW'
            self.lr = 5e-4
            self.weight_decay = 1e-5
            self.scheduler = 'CosineAnnealingLR'
            self.min_lr = 1e-6
            self.T_max = self.epochs

            self.aug_prob = 0.5
            self.mixup_alpha_real = 0.5
            self.mixup_alpha_pseudo = 0.5
            
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            
            self.use_pseudo_mixup = False  # pseudo lableでmixupするかどうか
            self.pseudo_mix_prob = 0.4  # mixupでpseudo lableを使う確率
            self.pseudo_conf_threshold = 0.5
            

            if self.debug:
                self.epochs = 2
                self.selected_folds = [0]
                self.batch_size = 4
                

In [7]:
cfg = CFG(mode="train", kaggle_notebook=False, debug=True)

In [8]:
utils_lib.set_seed(cfg.seed)

In [9]:
class BirdCLEFModelValidator:
    def __init__(self, cfg, df, datasets_lib, models_lib):
        self.cfg = cfg
        self.df = df
        self.datasets_lib = datasets_lib
        self.models_lib = models_lib
        self.label2index = {}
        self.index2label = {}

        self._load_taxonomy()
        self._load_spectrograms()

    def _load_taxonomy(self):
        taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        species_ids = taxonomy_df['primary_label'].tolist()
        self.cfg.num_classes = len(species_ids)
        self.index2label = {i: label for i, label in enumerate(species_ids)}
        self.label2index = {label: i for i, label in enumerate(species_ids)}

    def _load_spectrograms(self):
        print(f"Loading spectrograms from {self.cfg.spectrogram_npy}")
        self.spectrograms = np.load(self.cfg.spectrogram_npy, allow_pickle=True).item()
        print(f"Loaded {len(self.spectrograms)} spectrograms")

    def _get_val_df(self, fold):
        skf = StratifiedKFold(n_splits=self.cfg.n_fold, shuffle=True, random_state=self.cfg.seed)
        _, val_idx = list(skf.split(self.df, self.df['primary_label']))[fold]
        return self.df.iloc[val_idx].reset_index(drop=True)

    def _get_val_loader(self, val_df):
        val_dataset = self.datasets_lib.BirdCLEFDatasetFromNPY(
            val_df, self.cfg, self.spectrograms, mode='valid'
        )
        return DataLoader(val_dataset, batch_size=self.cfg.batch_size, shuffle=False,
                          num_workers=self.cfg.num_workers, pin_memory=True,
                          collate_fn=self.datasets_lib.collate_fn)

    def _load_model(self, model_path):
        device = torch.device(cfg.device)
        state = torch.load(model_path, map_location=device)
        model = self.models_lib.BirdCLEFModelForTrain(self.cfg).to(device)
        model.load_state_dict(state['model_state_dict'])
        return model.eval()

    def _predict_model(self, model, loader):
        device = torch.device(cfg.device)
        all_outputs, all_targets = [], []
        with torch.no_grad():
            for batch in tqdm(loader, desc="Predicting"):
                inputs = batch['melspec'].to(device)
                outputs = model(inputs)
                outputs = outputs[0] if isinstance(outputs, tuple) else outputs
                all_outputs.append(outputs.cpu().numpy())
                all_targets.append(batch['target'].numpy())
        return np.concatenate(all_outputs), np.concatenate(all_targets)

    def _evaluate_predictions(self, targets, outputs):
        probs = 1 / (1 + np.exp(-outputs))
        targets_bin = (targets >= 0.5).astype(int)
        auc = roc_auc_score(targets_bin, probs, average='macro')
        ap = average_precision_score(targets_bin, probs, average='macro')
        return {"auc": auc, "mAP": ap}

    def evaluate_model_dir(self, model_dir):
        full_dir = os.path.join(self.cfg.models_dir, model_dir)
        print(f"\n🔍 Evaluating model directory: {full_dir}")

        fold_preds = {}
        fold_targets = {}

        for fold in range(self.cfg.n_fold):
            model_path = os.path.join(full_dir, f"model_fold{fold}.pth")
            if not os.path.exists(model_path):
                print(f"⛔️ model_fold{fold}.pth not found in {model_dir}")
                continue

            val_df = self._get_val_df(fold)
            val_loader = self._get_val_loader(val_df)

            model = self._load_model(model_path)
            outputs, targets = self._predict_model(model, val_loader)

            fold_preds[fold] = outputs
            fold_targets[fold] = targets

            np.save(os.path.join(full_dir, f"predictions_fold{fold}.npy"), outputs)

            metrics = self._evaluate_predictions(targets, outputs)
            pd.DataFrame([metrics]).to_csv(
                os.path.join(full_dir, f"metrics_fold{fold}.csv"), index=False
            )

        for fold in fold_preds:
            preds = fold_preds[fold]
            targets = fold_targets[fold]
            np.save(os.path.join(full_dir, f"ensemble_pred_fold{fold}.npy"), preds)
            metrics = self._evaluate_predictions(targets, preds)
            pd.DataFrame([metrics]).to_csv(
                os.path.join(full_dir, f"ensemble_metrics_fold{fold}.csv"), index=False
            )

        print(f"\n✅ Finished evaluation for model_dir: {model_dir}")


In [ ]:
reload(learning_lib)
if __name__ == "__main__":
    print("\nLoading training data...")
    train_df = pd.read_csv(cfg.train_csv)

    model_dirs = [
    "baseline_fold0_7sec",
    ]
    validator = BirdCLEFModelValidator(cfg, train_df, datasets_lib, models_lib)
    # ====== 実行 ======
    for model_dir in model_dirs:
        validator.evaluate_model_dir(model_dir)



Loading training data...
Loading spectrograms from ../data/processed/baseline/birdclef2025_melspec_5sec_256_256.npy
Loaded 28564 spectrograms

🔍 Evaluating model directory: ../models/baseline_fold0_7sec
Found 5713 matching spectrograms for valid dataset out of 5713 samples


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:215: UserWarning: 
NVIDIA H100 PCIe with CUDA capability sm_90 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA H100 PCIe GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Predicting:   0%|          | 0/250 [00:00<?, ?it/s]